In [1]:
import tensorflow as tf
import numpy as np

In [2]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

#### Build Computation Graph

In [4]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        std = 1 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=std)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        return Z

In [5]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs", activation=tf.nn.relu)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
## can also use dense layer
# with tf.name_scope("dnn"):
#     hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
#     hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
#     logits = tf.layers.dense(hidden2, n_outputs, name="outputs", activation=tf.nn.relu)

In [7]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [8]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#### Execution Phase

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [12]:
n_epochs = 40
batch_size = 50

In [13]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for interaction in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            feed = {X: X_batch, y: y_batch}
            sess.run(training_op, feed_dict=feed)
        acc_train = accuracy.eval(feed_dict=feed)
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print("epoch %i, train acc %.5f, val acc %.5f"%(epoch, acc_train, acc_val))
    saver.save(sess, "outputs/final_model.ckpt")

epoch 0, train acc 0.82000, val acc 0.80800
epoch 1, train acc 0.78000, val acc 0.85100
epoch 2, train acc 0.86000, val acc 0.87720
epoch 3, train acc 0.92000, val acc 0.88600
epoch 4, train acc 0.92000, val acc 0.89640
epoch 5, train acc 0.92000, val acc 0.90300
epoch 6, train acc 0.94000, val acc 0.89940
epoch 7, train acc 0.92000, val acc 0.91460
epoch 8, train acc 0.90000, val acc 0.92020
epoch 9, train acc 0.92000, val acc 0.91820
epoch 10, train acc 0.98000, val acc 0.92640
epoch 11, train acc 0.92000, val acc 0.92880
epoch 12, train acc 0.88000, val acc 0.93720
epoch 13, train acc 0.94000, val acc 0.93920
epoch 14, train acc 0.94000, val acc 0.94160
epoch 15, train acc 1.00000, val acc 0.94140
epoch 16, train acc 0.94000, val acc 0.94660
epoch 17, train acc 0.92000, val acc 0.94540
epoch 18, train acc 0.94000, val acc 0.95380
epoch 19, train acc 0.98000, val acc 0.95580
epoch 20, train acc 0.92000, val acc 0.95420
epoch 21, train acc 0.96000, val acc 0.95480
epoch 22, train acc 

#### Test

In [14]:
with tf.Session() as sess:
    saver.restore(sess, "outputs/final_model.ckpt")
    Z = logits.eval(feed_dict={X: mnist.test.images})
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
acc_test

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from final_model.ckpt


0.9584